In [1]:
import requests
import os
import json
import pandas as pd

# ===================== Funciones de Scraping =====================

def cargar_url():
    base_url = "https://datos.gob.es/apidata/catalog/dataset/a10002983-mediciones-por-horas-de-contaminantes-atmosfericos-y-ozono-de-la-comunitat-valenciana-"
    urls = [f"{base_url}{year}?_sort=title&_pageSize=10&_page=0" for year in range(1994, 2025)]
    return urls

def obtener_anos_url(api_url):
    return api_url.split('-')[-1][:4]

def obtener_datos_url(api_url, year):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    try:
        response = requests.get(api_url, headers=headers)
        response.raise_for_status()
        data = response.json()
        download_folder = f"{year}_csv_files"
        if not os.path.exists(download_folder):
            os.makedirs(download_folder)
        
        def encontrar_descargar_csv(d):
            if isinstance(d, dict):
                for value in d.values():
                    encontrar_descargar_csv(value)
            elif isinstance(d, list):
                for item in d:
                    encontrar_descargar_csv(item)
            elif isinstance(d, str) and d.endswith(".csv"):
                print(f"URL de archivo CSV encontrado: {d}")
                descargar_csv(d, download_folder)

        encontrar_descargar_csv(data)
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener los datos de la API: {e}")

def descargar_csv(csv_url, download_folder):
    try:
        response = requests.get(csv_url, stream=True)
        response.raise_for_status()
        filename = csv_url.split("/")[-1]
        file_path = os.path.join(download_folder, filename)
        with open(file_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Archivo CSV descargado: {filename} en la carpeta {download_folder}")
    except requests.exceptions.RequestException as e:
        print(f"Error al descargar el archivo CSV desde {csv_url}: {e}")

def procesar_anos():
    urls = cargar_url()
    for url in urls:
        year = obtener_anos_url(url)
        print(f"Procesando datos para el año {year}...")
        obtener_datos_url(url, year)

# ===================== Procesamiento de Datos =====================

def procesar_csv():
    base_dir = os.getcwd()
    year_folders = [f for f in os.listdir(base_dir) if os.path.isdir(f) and f.endswith("_csv_files")]
    estaciones = ["ELX - AGROALIMENTARI", "ORIHUELA", "TORREVIEJA"]
    combined_data = pd.DataFrame()

    for folder in year_folders:
        print(f"Procesando la carpeta: {folder}")
        folder_path = os.path.join(base_dir, folder)
        csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

        for csv_file in csv_files:
            csv_path = os.path.join(folder_path, csv_file)
            print(f"Leyendo el archivo: {csv_file}")
            try:
                df = pd.read_csv(csv_path, encoding='utf-8', delimiter=';', on_bad_lines='skip')
                if 'NOM_ESTACION' not in df.columns:
                    print(f"El archivo {csv_file} no contiene la columna 'NOM_ESTACION'.")
                    continue
                df_filtered = df[df['NOM_ESTACION'].isin(estaciones)]
                if not df_filtered.empty:
                    combined_data = pd.concat([combined_data, df_filtered], ignore_index=True)
            except Exception as e:
                print(f"Error al procesar el archivo {csv_file}: {e}")

    if not combined_data.empty:
        output_folder = os.path.join(base_dir, "ESTACIONES_COMBINADAS_NO_LIMPIO")
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        output_file = os.path.join(output_folder, "estaciones_combinadas_NO_LIMPIO.csv")
        combined_data.to_csv(output_file, index=False, encoding='utf-8')
        print(f"Archivo combinado de estaciones guardado en: {output_file}")

# ===================== Limpieza de Datos =====================

def limpiezaAutomaticaPorEstacion(dataFrame):
    dataFrame.replace("-", pd.NA, inplace=True)
    columnasImportantes = ['COD_ESTACION', 'NOM_ESTACION', 'FECHA', 'HORA', 'SO2', 'CO', 'NO', 'NO2', 'NOx', 'O3']
    dataFrame = dataFrame[columnasImportantes]
    columnas_con_nulos = ['SO2', 'NO', 'NO2', 'NOx', 'O3']

    for columna in columnas_con_nulos:
        dataFrame[columna] = pd.to_numeric(dataFrame[columna], errors='coerce')
        dataFrame[columna].fillna(dataFrame[columna].mean(), inplace=True)

    dataFrame = dataFrame.dropna(axis=1, thresh=0.5 * len(dataFrame) * 0.5)
    return dataFrame

# ===================== Exportación a JSON y CSV =====================

def dataFrametoJSON(dataFrame, nombre):
    json_object = dataFrame.to_json(orient='records')
    filename = f"{nombre}-LIMPIO.json"
    with open(filename, 'w') as file:
        file.write(json_object)
    print(f"Archivo guardado como {filename}")

# ===================== Ejecución del Script =====================

if __name__ == "__main__":
    procesar_anos()
    procesar_csv()

    # Cargar datos procesados
    dataCompleto = pd.read_csv('./ESTACIONES_COMBINADAS_NO_LIMPIO/estaciones_combinadas_NO_LIMPIO.csv', decimal=",")
    estaciones = ["ELX - AGROALIMENTARI", "TORREVIEJA", "ORIHUELA"]
    conjuntos_datos = {}

    for estacion in estaciones:
        dataEstacion = dataCompleto[dataCompleto["NOM_ESTACION"] == estacion]
        dataEstacionLimpio = limpiezaAutomaticaPorEstacion(dataEstacion)
        conjuntos_datos[estacion] = dataEstacionLimpio

        # Exportar a JSON y CSV
        dataFrametoJSON(dataEstacionLimpio, estacion.replace(" ", "_"))
        dataEstacionLimpio.to_csv(f"{estacion.replace(' ', '_')}-Limpio.csv", sep=";", decimal=",", index=False)
        print(f"Datos limpios exportados para la estación {estacion}.")

Procesando datos para el año 1994...
URL de archivo CSV encontrado: https://dadesobertes.gva.es/dataset/13079cca-6ed6-4dec-b99a-788bcaad9ed8/resource/0a2ef8a1-9c8b-46cb-9d34-98a77ffe6807/download/contaminacion-atmosferica-y-ozono-promedios-por-horas_199409.csv
Archivo CSV descargado: contaminacion-atmosferica-y-ozono-promedios-por-horas_199409.csv en la carpeta 1994_csv_files
URL de archivo CSV encontrado: https://dadesobertes.gva.es/dataset/13079cca-6ed6-4dec-b99a-788bcaad9ed8/resource/98778330-da64-4c87-b536-542211fdb34e/download/contaminacion-atmosferica-y-ozono-promedios-por-horas_199404.csv
Archivo CSV descargado: contaminacion-atmosferica-y-ozono-promedios-por-horas_199404.csv en la carpeta 1994_csv_files
URL de archivo CSV encontrado: https://dadesobertes.gva.es/dataset/13079cca-6ed6-4dec-b99a-788bcaad9ed8/resource/cd67b5a1-a61c-4b94-a161-043993b42fa5/download/contaminacion-atmosferica-y-ozono-promedios-por-horas_199403.csv
Archivo CSV descargado: contaminacion-atmosferica-y-ozo

C:\Users\snkle\AppData\Local\Temp\ipykernel_21284\1259336905.py:78: DtypeWarning: Columns (18,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, encoding='utf-8', delimiter=';', on_bad_lines='skip')


Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_200707.csv
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_200708.csv
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_200709.csv
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_200710.csv
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_200711.csv
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_200712.csv
Procesando la carpeta: 2008_csv_files
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_200801.csv
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_200802.csv
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_200803.csv
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_200804.csv
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_200805.csv
Leyendo el archivo: contami

C:\Users\snkle\AppData\Local\Temp\ipykernel_21284\1259336905.py:78: DtypeWarning: Columns (10,11,12,17,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, encoding='utf-8', delimiter=';', on_bad_lines='skip')


Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_202004.csv
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_202005.csv
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_202006.csv
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_202007.csv
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_202008.csv
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_202009.csv
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_202010.csv
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_202011.csv
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_202012.csv
Procesando la carpeta: 2021_csv_files
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_202101.csv
Leyendo el archivo: contaminacion-atmosferica-y-ozono-promedios-por-horas_202102.csv
Leyendo el archivo: contami

...

Archivo guardado como TORREVIEJA-LIMPIO.json
Datos limpios exportados para la estación TORREVIEJA.


C:\Users\snkle\AppData\Local\Temp\ipykernel_21284\1259336905.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame.replace("-", pd.NA, inplace=True)
C:\Users\snkle\AppData\Local\Temp\ipykernel_21284\1259336905.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame[columna] = pd.to_numeric(dataFrame[columna], errors='coerce')
C:\Users\snkle\AppData\Local\Temp\ipykernel_21284\1259336905.py:106: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change 

Archivo guardado como ORIHUELA-LIMPIO.json
Datos limpios exportados para la estación ORIHUELA.


C:\Users\snkle\AppData\Local\Temp\ipykernel_21284\1259336905.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame.replace("-", pd.NA, inplace=True)
C:\Users\snkle\AppData\Local\Temp\ipykernel_21284\1259336905.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataFrame[columna] = pd.to_numeric(dataFrame[columna], errors='coerce')
C:\Users\snkle\AppData\Local\Temp\ipykernel_21284\1259336905.py:106: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change 

Archivo guardado como ALACANT_-_EL_PLA-LIMPIO.json
Datos limpios exportados para la estación ALACANT - EL PLA.
